#### Data 612 - Project 1<br>Christina Valore, Juliann McEachern, Rajwant Mishra<br>Due: June 11, 2019

# Global Baseline Predictors and RMSE

## Overview

In this first assignment, we'll attempt to predict ratings with very little information.  We'll first look at just raw averages across all (training dataset) users.  We'll then account for "bias" by normalizing across users and across items.   You'll be working with ratings in a user-item matrix, where each rating may be (1) assigned to a training dataset, (2) assigned to a test dataset, or (3) missing. 

Please code as much of your work as possible in R or Python.  You may use standard functions (e.g. from base R and the tidyverse).  Your project should be delivered in an R Markdown or a Jupyter notebook, then the notebook should be saved into a GitHub repository.  You should include a link to your GitHub repository in your assignment submission link. 

## Recommender System Data Selection

* Briefly describe the recommender system that you're going to build out from a business perspective, e.g. "This system recommends data science books to readers."
* Find a dataset, or build out your own toy dataset.  As a minimum requirement for complexity, please include numeric ratings for at least five users, across at least five items, with some missing data. 

## Data Preparation 

*  Load your data into (for example) an R or pandas dataframe, a Python dictionary or list of lists, (or another data structure of your choosing).  From there, create a user-item matrix. 
*  If you choose to work with a large dataset, you're encouraged to also create a small, relatively dense "user-item" matrix as a subset so that you can hand-verify your calculations.
*  Break your ratings into separate training and test datasets.


In [1]:
import pandas as pd
import numpy as np

#Import data from csv in github repository 
data = pd.read_csv('https://raw.githubusercontent.com/jemceach/612-group/master/project-1/Data/User%20_Mov%20.csv')

#view the imported data
data

,ID,Movie_ID,User_Name,Rating,Title,Content,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24
0,1,1,TheTopDawgCritic,8,Better than expected!,"For a B-grade film, this was surprisingly bett...",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,1,celestinoavilajr,8,Entertaining!,Good movie! Great plot! Edge of your seat twis...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,1,dave-mcclain,8,an expectedly good thriller,"Bad Samaritan"" R",151 is a thriller directed by actor-turned-pr...,"written by Brandon Boyce ""Wicker Park"" and st...",but this film is primarily an intricately plo...,a young man whose step-father brought him and...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,1,neener3707,7,"Done Before, But Not Too Bad, A Little Extra","Over all not too shabby, definitely better tha...",Im not sure what I want to say,because some of it was effective,and some of it wasnt. First of all,I like how they expanded on the typical story...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,1,krice23,8,Sold!,Original script. Believable cast. They make it...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,6,1,patrickc-76383,7,Impressive!,This movie takes you on an rollercoaster-like ...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,7,1,george.schmidt,7,Decent suspense thriller with one box of craz,"BAD SAMARITAN 2018 1/2 David Tennant, Robert S...",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,8,1,godsgirlfrombluford,9,Edge of your seat romp that keeps the viewer,Bad Samaritan is an edge of your seat romp tha...,lacking any conscience for the things he does...,taken and kept against her will.I felt for Se...,trying to save Katie. His desperation on-scre...,I wanted to jump through the screen and help ...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,9,1,claudio_carvalho,8,Surprisingly Great Thriller,"In Portland, the amateurish photographer Sean ...",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,10,1,Stars-and-popcorn,6,Bad Samaritan Plays it too Safe,Bad Samaritan is a movie that Ive been looking...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
#remove unnecessary columns, including blank columns
data=data.drop(data.columns.to_series()[4:25], axis=1)

#create small subset of the data for manual calculations, 5 rows to be exact
subst=data.iloc[:5,:]

#split the dataframe into training and test dataframes from rows 1-250 and from 250-687
train=data.iloc[:250, :]
test=data.iloc[250:, :]
train.head()


,ID,Movie_ID,User_Name,Rating
0,1,1,TheTopDawgCritic,8
1,2,1,celestinoavilajr,8
2,3,1,dave-mcclain,8
3,4,1,neener3707,7
4,5,1,krice23,8


In [11]:
test.head()

,ID,Movie_ID,User_Name,Rating
250,252,11,gbill-74877,8
251,253,11,atlasmb,8
252,254,11,steve-552-934420,1
253,255,11,dfranzen70,8
254,256,11,poetschn,5


In [3]:
#view the subsetted data
subst

,ID,Movie_ID,User_Name,Rating
0,1,1,TheTopDawgCritic,8
1,2,1,celestinoavilajr,8
2,3,1,dave-mcclain,8
3,4,1,neener3707,7
4,5,1,krice23,8


In [4]:
#view the end of the train data
train.tail()

,ID,Movie_ID,User_Name,Rating
245,247,11,zofos,5
246,248,11,radivel,9
247,249,11,Spikeopath,9
248,250,11,blackmambamark,8
249,251,11,kellyfacebook1,5


In [5]:
#view the end of the test data
test.tail()

,ID,Movie_ID,User_Name,Rating
682,684,6,bob-the-movie-man,4
683,685,6,lorriejinx,1
684,686,6,plargreg,1
685,687,6,luke-solly,1
686,688,6,viraltux,4


## RSME 
*  Using your training data, calculate the raw average (mean) rating for every user-item combination. 
*  Calculate the RMSE for raw average for both your training data and your test data.
*  Using your training data, calculate the bias for each user and each item.
*  From the raw average, and the appropriate user and item biases, calculate the baseline predictors for every user-item combination. 
*  Calculate the RMSE for the baseline predictors for both your training data and your test data.

## Summary 
*  Summarize your results